In [1]:
#Importing necessary libraries
import pandas as pd
import sqlite3

In [2]:
db = sqlite3.connect("/Users/apple/Desktop/database.sqlite")

In [3]:
# Using the pandas function 'read_sql_query'
# to read the results of a SQL query directly into a pandas DataFrame
def run_query(query):
    return pd.read_sql_query(query, db)

In [4]:
run_query("SELECT tbl_name FROM sqlite_master;")

,tbl_name
0,station
1,status
2,trip
3,weather


In [5]:
# Get first three rows of the 'trip' table
run_query("SELECT * FROM trip LIMIT 3;")

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4069,174,8/29/2013 9:08,2nd at South Park,64,8/29/2013 9:11,2nd at South Park,64,288,Subscriber,94114
1,4073,1067,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:42,San Francisco Caltrain 2 (330 Townsend),69,321,Subscriber,94703
2,4074,1131,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,317,Subscriber,94115


In [6]:
# Get only selected columns in the 'weather' table
run_query("SELECT date, mean_temperature_f, mean_wind_speed_mph FROM weather LIMIT 3;")

,date,mean_temperature_f,mean_wind_speed_mph
0,8/29/2013,68,11
1,8/30/2013,69,13
2,8/31/2013,64,15


In [7]:
# Longest bike trips from '2nd at South Park' station with ending station
query='''
SELECT start_station_name, end_station_name, duration FROM trip 
WHERE start_station_name = '2nd at South Park' 
ORDER BY duration DESC 
LIMIT 3;
'''
run_query(query)

,start_station_name,end_station_name,duration
0,2nd at South Park,Townsend at 7th,90213
1,2nd at South Park,Howard at 2nd,80917
2,2nd at South Park,Howard at 2nd,80883


In [8]:
# Compute average duration, renaming query result
run_query("SELECT AVG(duration) AS 'Average Duration' FROM trip;")

,Average Duration
0,1107.949846


In [9]:
# Get average duration by subscription group
query='''
SELECT subscription_type, AVG(duration) AS 'Average Duration'  FROM trip 
GROUP BY subscription_type;
'''
run_query(query)

,subscription_type,Average Duration
0,Customer,3951.761329
1,Subscriber,590.048856


In [10]:
# Checking the number of trips ended at the station ‘2nd at South Park‘
query = '''
SELECT COUNT(*) AS "Number of trips ended at the station 2nd at South Park"
FROM trip
WHERE end_station_name = '2nd at South Park';
'''
run_query(query)

,Number of trips ended at the station 2nd at South Park
0,16843


In [11]:
# Top five stations for the number of round trips
query = '''
SELECT start_station_name AS 'Station', COUNT(*) AS "Number of round trips"
FROM trip
WHERE start_station_name = end_station_name
GROUP BY start_station_name
ORDER BY COUNT(*) DESC
LIMIT 5;
'''
run_query(query)

,Station,Number of round trips
0,Embarcadero at Sansome,2092
1,Harry Bridges Plaza (Ferry Building),1733
2,University and Emerson,894
3,Market at 4th,724
4,2nd at Townsend,683


In [12]:
# Comparing the number of trips by city
query = '''
SELECT station.city AS "City", COUNT(*) AS "Count"
FROM trip 
INNER JOIN station
ON trip.start_station_name = station.name
GROUP BY station.city
ORDER BY COUNT(*) DESC;
'''
run_query(query)

,City,Count
0,San Francisco,592456
1,San Jose,37855
2,Mountain View,18167
3,Palo Alto,6773
4,Redwood City,3366


In [13]:
# Subquery example:
# Ten records that start at the station ‘Post at Kearney’ and have duration less than 160

# The important thing to remember when writing or reading subqueries is that
# the inner query inside the parentheses is executed first
# and afterwards the outer query is run on the results returned by the inner one.

query = '''
SELECT sub.*
FROM (SELECT * FROM trip
WHERE duration < 160) sub
WHERE start_station_name='Post at Kearney'
LIMIT 10;
'''
run_query(query)

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4749,138,8/29/2013 16:57,Post at Kearney,47,8/29/2013 16:59,Post at Kearney,47,408,Subscriber,94117
1,17227,140,9/10/2013 20:37,Post at Kearney,47,9/10/2013 20:39,Washington at Kearney,46,395,Subscriber,94133
2,20219,153,9/13/2013 12:12,Post at Kearney,47,9/13/2013 12:15,Market at 4th,76,525,Subscriber,94124
3,29176,67,9/20/2013 17:16,Post at Kearney,47,9/20/2013 17:17,Post at Kearney,47,555,Customer,98116
4,29181,115,9/20/2013 17:17,Post at Kearney,47,9/20/2013 17:19,Post at Kearney,47,345,Customer,
5,31420,156,9/23/2013 10:59,Post at Kearney,47,9/23/2013 11:01,Beale at Market,56,292,Subscriber,94115
6,31619,96,9/23/2013 13:37,Post at Kearney,47,9/23/2013 13:39,Market at Sansome,77,529,Subscriber,94608
7,41712,89,10/1/2013 15:07,Post at Kearney,47,10/1/2013 15:08,Post at Kearney,47,202,Subscriber,97214
8,43003,144,10/2/2013 14:03,Post at Kearney,47,10/2/2013 14:05,Commercial at Montgomery,45,520,Customer,94602
9,45978,120,10/4/2013 15:50,Post at Kearney,47,10/4/2013 15:52,Market at 4th,76,553,Subscriber,94102


In [14]:
#There are four connected tables: 1)Station info, 2)Status with timestamps, 3)Trips & 4)Weather
#Let's see all the columns and first 10 rows of the Station table
querytry='''
    SELECT *
    FROM station
    LIMIT 10;
'''
run_query(querytry)

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
5,7,Paseo de San Antonio,37.333798,-121.886943,15,San Jose,8/7/2013
6,8,San Salvador at 1st,37.330165,-121.885831,15,San Jose,8/5/2013
7,9,Japantown,37.348742,-121.894715,15,San Jose,8/5/2013
8,10,San Jose City Hall,37.337391,-121.886995,15,San Jose,8/6/2013
9,11,MLK Library,37.335885,-121.885660,19,San Jose,8/6/2013


<h1>For each given location/city, how many docks were installed, how many stations are there and what is the average capacity of each station?

In [15]:
query = '''
    SELECT city, 
    SUM(dock_count) AS total_capacity, 
    COUNT(name) AS station_count, 
    ROUND(SUM(dock_count)/COUNT(name), 2) AS average_capacity_per_station
    FROM station
    GROUP BY city
    ORDER BY station_count DESC;
'''
run_query(query)

,city,total_capacity,station_count,average_capacity_per_station
0,San Francisco,665,35,19.0
1,San Jose,264,16,16.0
2,Redwood City,115,7,16.0
3,Mountain View,117,7,16.0
4,Palo Alto,75,5,15.0


In [16]:
<h1>Per city and date, what is the dock and station count?

Object `count` not found.


In [ ]:
<h1>Per city and date, what is the dock and station count

<h1>Per city and date, what is the dock and station count

In [17]:
query= '''
    SELECT city,
           CASE
           -- m/d/yyyy
           WHEN (length(installation_date) = 8 AND substr(installation_date,2,1) = '/') 
           THEN substr(installation_date,5,4)||'-0'||substr(installation_date,1,1)||'-0'||substr(installation_date,3,1)
           -- m/dd/yyyy
           WHEN (length(installation_date) = 9 AND substr(installation_date,2,1) = '/') 
           THEN substr(installation_date,6,4)||'-0'||substr(installation_date,1,1)||'-'||substr(installation_date,3,2)
           -- mm/d/yyyy
           WHEN (length(installation_date) = 9 AND substr(installation_date,3,1) = '/') 
           THEN substr(installation_date,6,4)||'-0'||substr(installation_date,1,2)||'-'||substr(installation_date,4,1)
           -- mm/dd/yyyy
           WHEN (length(installation_date) = 10 AND substr(installation_date,3,1) = '/')
           THEN substr(installation_date,7,4)||'-'||substr(installation_date,1,2)||'-'||substr(installation_date,4,2)
           ELSE installation_date
           END AS installed_date, 
           SUM(dock_count) AS total_dock_ct,
           COUNT(name) AS station_count
    FROM station
    GROUP BY 1,2
    ORDER BY 2,1;
'''
run_query(query)

,city,installed_date,total_dock_ct,station_count
0,San Jose,2013-08-05,83,5
1,San Jose,2013-08-06,87,5
2,San Jose,2013-08-07,64,4
3,Redwood City,2013-08-12,60,4
4,Palo Alto,2013-08-14,64,4
5,Mountain View,2013-08-15,46,2
6,Palo Alto,2013-08-15,11,1
7,Redwood City,2013-08-15,40,2
8,Mountain View,2013-08-16,45,3
9,San Francisco,2013-08-19,94,6


<h1>From above query's result, it seems like most installations occurred on August (of 2013). Let's find exact number for AUG13 installments and the rest

In [18]:
query = '''
    WITH t1 AS (SELECT city,
                       CASE
                       -- m/d/yyyy
                       WHEN (length(installation_date) = 8 AND substr(installation_date,2,1) = '/') 
                       THEN substr(installation_date,5,4)||'-0'||substr(installation_date,1,1)||'-0'||substr(installation_date,3,1)
                       -- m/dd/yyyy
                       WHEN (length(installation_date) = 9 AND substr(installation_date,2,1) = '/') 
                       THEN substr(installation_date,6,4)||'-0'||substr(installation_date,1,1)||'-'||substr(installation_date,3,2)
                       -- mm/d/yyyy
                       WHEN (length(installation_date) = 9 AND substr(installation_date,3,1) = '/') 
                       THEN substr(installation_date,6,4)||'-0'||substr(installation_date,1,2)||'-'||substr(installation_date,4,1)
                       -- mm/dd/yyyy
                       WHEN (length(installation_date) = 10 AND substr(installation_date,3,1) = '/') 
                       THEN substr(installation_date,7,4)||'-'||substr(installation_date,1,2)||'-'||substr(installation_date,4,2)
                             ELSE installation_date
                       END AS installed_date, 
                       SUM(dock_count) AS total_dock_ct,
                       COUNT(name) AS station_count
                FROM station
                GROUP BY 1,2
                ORDER BY 2,1)
    SELECT CASE 
        WHEN month = '2013-08-01'
        THEN 'AUG13'
        WHEN month > '2013-08-01'
        THEN 'after_AUG13'
        ELSE 'before_AUG13'
        END AS installation_month,
        SUM(total_dock_ct) AS dock_ct,
        SUM(station_count) AS station_ct
    FROM (SELECT DATE(installed_date, 'start of month') as month,
                 total_dock_ct, 
                 station_count
         FROM t1
         ) AS innerquery
    GROUP BY 1;
'''
run_query(query)

,installation_month,dock_ct,station_ct
0,AUG13,1150,64
1,after_AUG13,86,6


<h1>For each given station, on average how many bikes were available vs docks? And which station had the least bicycles and most docks available on average (or was the busiest station)?

In [19]:
query = '''
     SELECT ROUND(AVG(status.bikes_available),2) AS avg_available_bikes, 
            ROUND(AVG(status.docks_available),2) AS free_dock_count,
            station.dock_count AS max_dock_capacity,
            station.name
     FROM status
     INNER JOIN station
     ON status.station_id = station.id
     GROUP BY name
     ORDER BY 1, 2 DESC
     LIMIT 10;
'''
run_query(query)

KeyboardInterrupt: 

In [ ]:
query = 

run_query(query)

<h1>For each given station, on average how many bikes were available vs docks? And which station had the least bicycles and most docks available on average (or was the busiest station)?

In [ ]:
query = 

run_query(query)

In [ ]:
query = 

run_query(query)

In [ ]:
query = 

run_query(query)

In [ ]:
query = 

run_query(query)

In [ ]:
query = 

run_query(query)

In [ ]:
query = 

run_query(query)

In [ ]:
query = 

run_query(query)

In [ ]:
query = 

run_query(query)